In [12]:
import numpy as np
from functools import wraps
from time import time
from scipy.special import binom
import matplotlib.pyplot as plt

# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot
jtplot.style()

def timing(f):
    # Create timing wrapper
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec'% (f.__name__, args, kw, te-ts))
        return result
    return wrap

In [342]:

class binomial_tree_european:

    '''
    Binomial asset tree model with different options.

    We need to know several things:
    u, d, S0, T, N, r, K, option
    '''
    def __init__(self, params):

        self.__dict__.update(params)    # load model parameters
        self.dt = self.T/self.N         # Compute time step
        self.disc = (1+self.r)           # Compute discount rate
        self.p = (self.disc - self.d) / (self.u - self.d)  # Compute risk neutral probability
        self.delta0 = self.calculate_delta0()
        self.V0 = self.calculate_V0()
        self.X0 = self.calculate_V0()

    def calculate_Sn(self, n):
        # Compute potential price outcomes
        return self.S0 * self.d ** (np.arange(n, -1, -1)) * self.u ** (np.arange(0, n + 1, 1))

    def VN(self):
        # Compute asset prices at maturity T = N
        if self.option == 'put':
            return np.maximum(self.K - self.calculate_Sn(self.N), np.zeros(self.N+1))
        else:
             return np.maximum(self.calculate_Sn(self.N)-self.K, np.zeros(self.N+1))

    def calculate_V0(self):
        # Compute initial asset price through tree at time n
        C = self.VN()
        for i in np.arange(self.N, 0, -1):
          C =  (self.p * C[1:i+1] + (1-self.p) * C[0:i])/self.disc

        return C[0]

    def calculate_Vn(self, n):
        # Compute initial asset price through tree at time n
        C = self.VN()
        for i in np.arange(self.N, n, -1):
            C = (self.p * C[1:i + 1] + (1 - self.p) * C[0:i])/self.disc

        return C

    def calculate_delta0(self):
        # Compute risk neutal share holding
        self.d0 = -np.diff(self.calculate_Vn(1))[0]/((self.u-self.d)*self.S0)
        return self.d0

    def calculate_deltan(self, n):
        # Calculate
        return -np.diff(self.calculate_Vn(n+1))/np.diff(self.calculate_Sn(n+1))

    def calculate_X0(self):
        # Calculate X0
        return sum(self.disc * self.calculate_Vn(1) * np.array([1-self.p, self.p]))
    
    def calculateXN(self):
        # Compute XN
        
        pass

In [343]:

u = 2

params = {'S0': 4,
            'r': 0.25,
            'u': u,
            'd': 1/u,
            'K': 5,
            'N': 3,
            'T': 1,
            'option': 'put'}

# create random model
m = binomial_tree_european(params)

n = 3
Sn = m.calculate_Sn(n)
Vn = m.calculate_Vn(n)
probs = (1-m.p)**(np.arange(n, -1, -1)) * (m.p)**(np.arange(0, n+1, 1))
bs = binom(n, np.arange(n+1))

# C
Sbar =  sum(bs * probs * Sn) * m.disc ** n
Vbar = sum(bs * probs * Vn) * m.disc ** n

print("V0 = ", np.round(m.V0, 3), " and discounted average o Vn = ", np.round(Vbar, 3))
print("S0 = ", m.S0, " and discounted average o Vn = ", np.round(Sbar, 3))

V0 =  0.864  and discounted average o Vn =  3.296
S0 =  4  and discounted average o Vn =  15.259


We now want compute the wealth growth, but we must be wary that we always have n+1 ratios, and n+2 underlying prices

Let n = 3, then the 5 elements of S correspond to HHHH HHHT HHTT HTTT TTTT, the stocks ratios on the other hand go as HHH HHT HTT TTT.
We need to construct a tree 

In [344]:
X1 = m.delta0 * m.calculate_Sn(1) + m.disc * (m.X0 - m.delta0 * m.S0)

n = 1
S2 = m.calculate_Sn(n+1)
S1 = m.calculate_Sn(n)
d1 = m.calculate_deltan(n)

for i in range(n+1):
    print(d1[i] * S2[i:i+2] + (m.disc)**(-1) * (X1 - S1*d1))
    

[0.024 3.104]
[-0.176  2.304]


In [345]:
m.calculate_Vn(1)

array([1.68, 0.48])

In [346]:
X1

array([0.48, 1.68])

In [339]:
m.calculate_Vn(1)

array([1.68, 0.48])

In [330]:
 m.calculate_Sn(3)

array([ 0.5,  2. ,  8. , 32. ])

In [331]:
C = m.VN()
print(C)
for i in np.arange(m.N, 0, -1):
    C = ((m.p)* C[1:i+1] + (1-m.p) * C[0:i])
    print(C)

[4.5 3.  0.  0. ]
[4.6875 1.875  0.    ]
[4.1015625 1.171875 ]
[3.29589844]


In [322]:
X1

array([1.359872, 2.588672])

In [323]:
m.p * m.calculate_Vn(1)[0] + (1-m.p) * m.calculate_Vn(1)[1]

1.4745599999999999

In [304]:
(X1[0]*m.p + X1[1]*(1-m.p))/m.disc

1.1550244863830923

In [297]:
X1

array([0.38376195, 1.52552101])

## We have computed all the values on average, but let's compute it for a trajectory